In [2]:
def instagram_login(username, password):
    
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time

    driver = webdriver.Firefox()
    driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
    while True:
        try:
            elem = driver.find_element_by_name('username')
            elem.clear()
            elem.send_keys(username)
            break
        except:
            pass

    elem = driver.find_element_by_name('password')
    elem.clear()
    elem.send_keys(password)

    elem.send_keys(Keys.RETURN)

    while True:
        try:
            driver.find_element_by_xpath("//button[@class='aOOlW   HoLwm ']").click()
            break
        except:
            pass


In [3]:
def word_frequency(string, min_freq, ignore_list):
    if(str(type(string)) == "<class 'list'>"):
        string = ' '.join(string)
        
    word_freq = []

    str1 = string.split()
    str2 = list(set(str1))

    for word in str2:
        if str1.count(word) >= min_freq and word not in ignore_list:
            word_freq.append((word, str1.count(word)))
            
    word_freq = sorted(word_freq, key = lambda x: x[1], reverse = True)
    return word_freq


In [4]:
def get_posts_url_list(page_name, number_of_posts):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time

    driver = webdriver.Firefox()
    driver.get('https://www.instagram.com/' + page_name)

    posts_url_list = []

    while len(posts_url_list) < number_of_posts:

        posts = driver.find_elements_by_xpath("//div[@class='v1Nh3 kIKUG  _bz0w']")

        for post in posts:
            post_url = post.find_element_by_css_selector('a').get_attribute("href")
            if post_url not in posts_url_list and len(posts_url_list) < number_of_posts:
                posts_url_list.append(post_url)

        elem = driver.find_element_by_tag_name('html')    
        elem.send_keys(Keys.END)
        time.sleep(2)
    
    driver.close()
        
    return posts_url_list


In [5]:
def get_comments(posts_url_list):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time

    driver = webdriver.Firefox()
    
    comments_list, users_list = [], []

    for post in posts_url_list:
        driver.get(post)
        count = 0
        while True:
            try:
                elem = driver.find_element_by_xpath("//span[@aria-label='Load more comments']")
                elem.click()
            except:
                try:
                    elem = driver.find_element_by_xpath("//button[@class='Z4IfV _0mzm- sqdOP yWX7d        ']")
                    elem.click()
                except:
                    count += 1
                    pass
            if count == 1000:
                break

        while True:
            try:
                comments = driver.find_elements_by_class_name("C4VMK")
                break
            except:
                pass

        for c in comments:
            count = 0
            while count<10:
                try:
                    comment = c.find_element_by_css_selector('span').get_attribute("textContent")
                    user = c.find_element_by_class_name("TlrDj").get_attribute("textContent")
                    comments_list.append(comment)
                    users_list.append(user)
                    break
                except:
                    count += 1
                    pass
            if count == 10:
                print('Error')
        print('---------------------')
            
            
    driver.close()
        
    return users_list, comments_list


In [6]:
def comments_to_csv(users_list, comments_list, file_name):
    import pandas as pd
    
    df = pd.DataFrame({"User": users_list, "comment": comments_list})
    df.to_csv(file_name + ".csv", index = False)

In [7]:
def comments_from_csv(file_name):
    import pandas as pd
    
    data = pd.read_csv(file_name + ".csv") 
    data = data.values.tolist()

    users_list = list(x[0] for x in data)
    comments_list = list(x[1] for x in data)

    return users_list, comments_list

In [8]:
def total_word_count(list_of_strings):
    count = 0
    for string in list_of_strings:
        count += len(string)
        
    return count

In [9]:
def ignore_list_creator(csv_list, pre_address, ignore_list):
    total_list = []
    for item in csv_list:
        users_list, comments_list = comments_from_csv(pre_address + item)
        list_of_word_freq = word_frequency(comments_list, total_word_count(comments_list)/10000, [])
        total_list += list(map(lambda x: x[0], list_of_word_freq))
        
    new_ignore_list = word_frequency(total_list, 2, [])
    
    ignore_list += list(map(lambda x: x[0], new_ignore_list))
    
    return ignore_list    

In [64]:
page_list = ['90tv.official', 'hassan_reyvandi', 'masih.alinejad', 'ebi', 'chikhobe', 'sooriland', 'film_bazzan',\
            'varzesh3ir', 'mr.taster', 'art.fatemehebadi', 'homayounshajarian', 'rambodjavan1']

for page in page_list:
    posts_url_list = get_posts_url_list(page, 30)
    users_list, comments_list = get_comments(posts_url_list)
    comments_to_csv(users_list, comments_list, 'InstagramData/' + page)

In [11]:
page_list = ['90tv.official', 'hassan_reyvandi', 'masih.alinejad', 'ebi', 'chikhobe', 'sooriland', 'film_bazzan',\
            'mr.taster', 'art.fatemehebadi', 'rambodjavan1']

ignore_list = ignore_list_creator(page_list, 'InstagramData/', [])

In [23]:
## Sample

users_list, comments_list = comments_from_csv('InstagramData/masih.alinejad')
lst = word_frequency(comments_list, total_word_count(comments_list)/10000, ignore_list)
for i in lst:
    print(i)

('مسیح', 209)
('پدر', 152)
('استادیوم', 116)
('زنان', 76)
('دروغ', 64)
('مرگ', 64)
('خاک', 62)
('ملت', 53)
('مراسم', 51)
('آزادی', 50)
('حکومت', 48)
('خودشون', 46)
('❤', 42)
('لعنت', 42)
('حجاب', 41)
('کاری', 41)
('داریوش', 40)
('غیرت', 38)
('نظام', 36)
('برن', 36)
('خون', 34)
('حقوق', 34)
('بگه', 33)
('اسلامی', 32)
('خودشو', 32)
('دخترت', 32)
('جمهوری', 31)
('!', 31)
('بانوان', 31)
('دخترش', 30)
('علینژاد', 30)
('بیرون', 29)
('دادن', 29)
('دختری', 29)
('هستند', 29)
('رفت', 27)
('مشکل', 27)
('دختران', 27)
('رژیم', 27)
('روانی', 26)
('بیچاره', 26)
('انقلاب', 26)
('بگو', 26)
('پدرش', 26)
('معلومه', 26)
('زندان', 25)
('زور', 25)
('داد', 25)
('ننگ', 25)


In [ ]:
page_list = ['90tv.official', 'hassan_reyvandi', 'masih.alinejad', 'ebi', 'chikhobe', 'sooriland', 'film_bazzan',\
            'varzesh3ir', 'mr.taster', 'art.fatemehebadi', 'homayounshajarian', 'rambodjavan1']

page_list = ['sooriland']

for page in page_list:
    posts_url_list = get_posts_url_list(page, 1)
    users_list, comments_list = get_comments(posts_url_list)
    comments_to_csv(users_list, comments_list, 'InstagramData/' + page)

0


In [9]:
def get_comments(posts_url_list):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time

    driver = webdriver.Firefox()
    
    comments_list, users_list = [], []

    for post in posts_url_list:
        print(len(comments_list))
        driver.get(post)
        count = 0
        while True:
            try:
                elem = driver.find_element_by_xpath("//span[@aria-label='Load more comments']")
                elem.click()
            except:
                try:
                    elem = driver.find_element_by_xpath("//button[@class='Z4IfV _0mzm- sqdOP yWX7d        ']")
                    elem.click()
                except:
                    count += 1
                    pass
            if count == 1000:
                break

            while True:
                try:
                    comments = driver.find_elements_by_class_name("C4VMK")
                    break
                except:
                    pass

            for c in comments:
                count = 0
                while count<10:
                    try:
                        comment = c.find_element_by_css_selector('span').get_attribute("textContent")
                        user = c.find_element_by_class_name("TlrDj").get_attribute("textContent")
                        if comment not in comments_list:
                            comments_list.append(comment)
                            users_list.append(user)
                        break
                    except:
                        count += 1
                        pass
#                 if count == 10:
#                     print('Error')
#             print('---------------------')
            
            
    driver.close()
        
    return users_list, comments_list
